In [1]:
# %pip install --upgrade langchain langsmith langchain_openai

In [2]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OpenAI keys from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Get Anyscale and Together keys from .env file
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")
os.environ["ANYSCALE_API_KEY"] = os.getenv("ANYSCALE_API_KEY")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "CodeLlama Demo"

### Set up our chain

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser

# Set up the system template
system_template = """
You are an expert programmer that helps to write Python code based on the user request, with concise explanations. Don't be too verbose.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Set up the human template with a variable for the request
human_template = """
{request}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

output_parser = StrOutputParser()

In [4]:
# Set up several different models using the same ChatOpenAI
from langchain_openai import ChatOpenAI

# Use OpenAI directly
MODEL_OPENAI = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Use CodeLlama with Together
MODEL_CODELLAMA_TOGETHER = ChatOpenAI(
    openai_api_base="https://api.together.xyz",
    api_key=os.getenv("TOGETHER_API_KEY"),
    model="codellama/CodeLlama-70b-Instruct-hf",
    temperature=0,
)

# Use CodeLlama with Anyscale
MODEL_CODELLAMA_ANYSCALE = ChatOpenAI(
    openai_api_base="https://api.endpoints.anyscale.com/v1",
    api_key=os.getenv("ANYSCALE_API_KEY"),
    model="codellama/CodeLlama-70b-Instruct-hf",
    temperature=0,
)

In [9]:
# Create a runnable with our chosen model
# model = MODEL_OPENAI
# model = MODEL_CODELLAMA_TOGETHER
model = MODEL_CODELLAMA_ANYSCALE

code_writing_runnable = prompt | model | output_parser

### Invoke our chain with a request

In [10]:
feature_request = """
Create a function that takes a list of strings as input and returns a list of the lengths of those strings. For example, if the input is ["hello", "world"], the function should return [5, 5].
"""

code_writing_runnable.invoke({"request": feature_request})

'\n\nHere is a simple function that takes a list of strings as input and returns a list of the lengths of those strings:\n\n```\ndef get_lengths(strings):\n    lengths = []\n    for string in strings:\n        lengths.append(len(string))\n    return lengths\n\n# Example usage:\ninput_list = ["hello", "world"]\noutput_list = get_lengths(input_list)\nprint(output_list)  # Output: [5, 5]\n```\n\nIn this function, we iterate over each string in the input list, calculate its length using the `len()` function, and append the length to a new list. Finally, we return the list of lengths.'